inspuired by
https://www.tensorflow.org/tutorials/keras/text_classification_with_hub
https://www.tensorflow.org/tutorials/keras/text_classification
https://developers.google.com/machine-learning/guides/text-classification

In [1]:
# Todo control logging with debug
import logging
import argparse 
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()
%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data-ignored/imdb/'
args.val_fraction = 0.25
args.vocab_size = 2500
args.small_vocab_size = 250
args.epochs = 50
args.batch_size = 32

Path(args.data_folder).mkdir(parents=True, exist_ok=True)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

ds, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True, data_dir=args.data_folder)
train_ds_len= tf.data.experimental.cardinality(ds['train']).numpy()
test_ds_len= tf.data.experimental.cardinality(ds['test']).numpy() 
print(train_ds_len)
print(test_ds_len)
for d in ds['train'].take(1):
    print(d)
    
# train_dataset = ds['train'].batch(args.batch_size)
train_dataset = ds['train']
val_dataset = ds['test'].take(int(args.val_fraction * (train_ds_len + test_ds_len)))
test_dataset = ds['test'].skip(int(args.val_fraction * (train_ds_len + test_ds_len)))

2023-07-31 12:11:45,329 : INFO : No config specified, defaulting to config: imdb_reviews/plain_text
2023-07-31 12:11:45,331 : INFO : Load dataset info from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0
2023-07-31 12:11:45,333 : INFO : Reusing dataset imdb_reviews (./data-ignored/imdb/imdb_reviews/plain_text/1.0.0)
2023-07-31 12:11:45,390 : INFO : Constructing tf.data.Dataset imdb_reviews for split None, from ./data-ignored/imdb/imdb_reviews/plain_text/1.0.0


Version:  2.13.0
Eager mode:  True
GPU is NOT AVAILABLE
25000
25000
(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


2023-07-31 12:11:45.414297: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### 1. Bag of words from scratch

In [2]:
class OneTextPreprocessing(tf.keras.layers.Layer):
    def __init__(self):
        super(OneTextPreprocessing, self).__init__()
    
    def call(self, inputs):
        # inputs shape: B, Ragged texts
        # print(f"Type inputs: {type(inputs)}")
        # print(f"Inputs shape: {inputs.shape}")
        batch_size = tf.shape(inputs)[0]
        outputs = tf.TensorArray(dtype=tf.string, size=0, dynamic_size=True)
        for i in tf.range(batch_size):
            string_tensor = inputs[i]
            byte_string = string_tensor.numpy()
            # print(f"byte_string: {byte_string}")
            processed = ''.join(c for c in byte_string.decode('UTF-8').lower() if c.isalnum() or c == " ")
            # print(f"type(processed): {type(processed)}")
            outputs = outputs.write(i, processed)
        stacked_output = outputs.stack()
        # print(f"type(stacked_output): {type(stacked_output)}")
        # print(f"tf.shape(stacked_output): {tf.shape(stacked_output)}")
        # outputs shape: B, Ragged texts
        return stacked_output

one_text_preprocessing = OneTextPreprocessing()

# for d in train_dataset.batch(args.batch_size).take(1):
for d in train_dataset.batch(2).take(1):
    one_processed = one_text_preprocessing(d[0])
    print(f"one_processed layer: {one_processed}")

one_model = keras.models.Sequential()
one_model.add(one_text_preprocessing)
for d in train_dataset.batch(2).take(1):
    one_processed = one_model(d[0])
    print(f"one_processed model: {one_processed}")

one_processed layer: [b'this was an absolutely terrible movie dont be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movies ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the columbian rebels were making their cases for revolutions maria conchita alonso appeared phony and her pseudolove affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actors like christopher walkens good name i could barely sit through it'
 b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the sette and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development wa

2023-07-31 12:11:50.000623: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-07-31 12:11:50.026794: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [3]:
class TwoTokenizer(tf.keras.layers.Layer):
    def __init__(self):
        super(TwoTokenizer, self).__init__()
    
    def call(self, inputs):
        # inputs shape: B, Ragged texts
        # print(f"Type inputs: {type(inputs)}")
        # print(f"Inputs shape: {inputs.shape}")
        batch_size = tf.shape(inputs)[0]
        tokens_flat_tensor_array = tf.TensorArray(dtype=tf.string, size=0, dynamic_size=True)
        str_len_tensor_array = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
        start_len = 0
        for i in tf.range(batch_size):
            string_tensor = inputs[i]
            byte_string = string_tensor.numpy()
            # print(f"byte_string: {byte_string}")
            string = byte_string.decode('UTF-8')
            tokens = string.split()
            for iw in tf.range(len(tokens)):
                tokens_flat_tensor_array = tokens_flat_tensor_array.write(start_len + iw, tokens[iw])
            str_len_tensor_array = str_len_tensor_array.write(i, len(tokens))
            start_len += len(tokens)
        
        ragged_tensor = tf.RaggedTensor.from_row_lengths(
                values=tokens_flat_tensor_array.stack(),
                row_lengths=str_len_tensor_array.stack())
        # print(f"type(ragged_tensor): {type(ragged_tensor)}")
        # print(f"tf.shape(ragged_tensor): {tf.shape(ragged_tensor)}")
        # outputs shape: Ragged tensor: B, Tokens
        return ragged_tensor
        
two_tokenizer = TwoTokenizer()

# for d in train_dataset.batch(args.batch_size).take(1):
for d in train_dataset.batch(2).take(1):
    two_processed = two_tokenizer(d[0])
    print(f"two_processed layer: {two_processed}")

two_model = keras.models.Sequential()
two_model.add(one_text_preprocessing)
two_model.add(two_tokenizer)
for d in train_dataset.batch(2).take(1):
    two_processed = two_model(d[0])
    print(f"two_processed model: {two_processed}")

two_processed layer: <tf.RaggedTensor [[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie.', b"Don't",
  b'be', b'lured', b'in', b'by', b'Christopher', b'Walken', b'or',
  b'Michael', b'Ironside.', b'Both', b'are', b'great', b'actors,', b'but',
  b'this', b'must', b'simply', b'be', b'their', b'worst', b'role', b'in',
  b'history.', b'Even', b'their', b'great', b'acting', b'could', b'not',
  b'redeem', b'this', b"movie's", b'ridiculous', b'storyline.', b'This',
  b'movie', b'is', b'an', b'early', b'nineties', b'US', b'propaganda',
  b'piece.', b'The', b'most', b'pathetic', b'scenes', b'were', b'those',
  b'when', b'the', b'Columbian', b'rebels', b'were', b'making', b'their',
  b'cases', b'for', b'revolutions.', b'Maria', b'Conchita', b'Alonso',
  b'appeared', b'phony,', b'and', b'her', b'pseudo-love', b'affair',
  b'with', b'Walken', b'was', b'nothing', b'but', b'a', b'pathetic',
  b'emotional', b'plug', b'in', b'a', b'movie', b'that', b'was', b'devoid',
  b'of', b'any', b'rea

2023-07-31 12:11:52.926966: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2023-07-31 12:11:52.989482: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [21]:
from collections import Counter 
import functools

class ThreeVectorizer(tf.keras.layers.Layer):
    def __init__(self, token_to_int: dict):
        super(ThreeVectorizer, self).__init__()
        self._token_to_int = token_to_int
        self._vocab_size = len(self._token_to_int) + 1
    
    def call(self, inputs):  
        batch_size = tf.shape(inputs)[0]
        outputs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        for i in tf.range(batch_size):
            tokens = inputs[i]
            array_string = tf.TensorArray(dtype=tf.float32, size=self._vocab_size)
            array_string.unstack(tf.zeros(self._vocab_size))
            for token in tokens:
                idx = self._token_to_int.get(token.numpy(), tf.constant(0))
                array_string = array_string.write(idx, array_string.read(idx) + 1.0)
            outputs = outputs.write(i, array_string.stack())
        return outputs.stack()

    @classmethod
    @functools.lru_cache(maxsize=10)
    def from_train_data(cls, train_ds, vocab_size=args.small_vocab_size, batch_size=args.batch_size, take=5):
        _one_text_preprocessing = OneTextPreprocessing()
        _two_tokenizer = TwoTokenizer()
        _preproc_mdl = keras.models.Sequential()
        _preproc_mdl.add(_one_text_preprocessing)
        _preproc_mdl.add(_two_tokenizer)

        _counter = Counter()
        for d in train_ds.batch(batch_size).take(take):
            _processed = _preproc_mdl(d[0])
            for b in _processed:
                _counter.update(b.numpy().tolist())

        # +-1 cause we need space for unknown tokens with 0 index
        _token_dict = {k: tf.cast(i + 1, tf.int32) for i, (k, _) in enumerate(_counter.most_common(vocab_size - 1))}
        return cls(_token_dict)

three_vectorizer = ThreeVectorizer.from_train_data(train_dataset)

three_model = keras.models.Sequential()
three_model.add(one_text_preprocessing)
three_model.add(two_tokenizer)
three_model.add(three_vectorizer)
for d in train_dataset.batch(2).take(1):
    three_processed = three_model(d[0])
    print(f"three_processed model: {three_processed}")

2023-07-31 16:10:13.355737: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


two_processed model: [[43.  2.  1.  2.  1.  0.  1.  3.  5.  1.  2.  2.  0.  1.  0.  1.  3.  0.
   3.  2.  0.  0.  2.  0.  0.  2.  0.  1.  0.  2.  0.  0.  1.  0.  0.  0.
   1.  0.  0.  0.  0.  1.  2.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.  0.
   3.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.
   0.  1.  0.  0.  2.  2.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  2.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.

2023-07-31 16:10:14.209751: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
